In [32]:
import json
import codecs
import pprint
import re
import os
import sys
import csv
import glob
import re
import json
import pandas as pd
from pprint import pprint
from pandas.io.json import json_normalize
import cPickle as pickle
import matplotlib.pyplot as plt
from IPython.display import display, HTML
%matplotlib inline
from bs4 import BeautifulSoup
import requests
import string
from collections import Counter


#nlp
import enchant
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

In [2]:
with open('./data/processed/unique_all_tweets_filtered.pickle', 'rb') as f:
    data = pickle.load(f)

# Text Preprocessing

### Text Replacement Dictionaries
First load and generate dictionaries for replacing twitter and internet specific slang

In [3]:
#load the twitter slang replacements which have been generated by analyzing the text
with open('./data/twitterSlang.csv','r') as f:
    reader = csv.reader(f,delimiter=',')
    slangTwitter = {row[0]:row[1] for row in reader}

In [4]:
#Build a slang dictionary based on the website
#In total 1461 slang words are described on this side.
#The following lines download the content, parse the HTML table and put all information into a python dictionary 
response = requests.get('http://www.webopedia.com/quick_ref/textmessageabbreviations.asp').content
soup = BeautifulSoup(response,'html.parser')
slangDict = {row.select('td')[0].getText().strip().lower():row.select('td')[1].getText().strip().lower() for row in soup.select('table tr') if len(row.select('td'))>1}

In [5]:
apostropheDictionary = {
    "'s":' is',
    "'t":' not',
    "'re":' are',
    "'d":" had",
    "'ll":" will",
    "'m":" am",
    "'ve":" have",
    "im":" i am",
    "i've":" i have"
}

In [6]:
#taken from wikipedia https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
with open('./data/english_contractions.json','r') as f:
    contractions = json.load(f)

## Define the Different Preprocessing Functions

In [48]:
def removeLinebreaks(tweet):
    return tweet.replace('\n', ' ').replace('\r', '').rstrip().strip()

rpt_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE)
def removeDuplicatedCharacters(tweet):
    """
    E.g. "i am happppy" => "i am happy"
    Even if not all characters are removed, like "hallo" => "halo", the spelling correction is applied to get a true word
    """
    return rpt_regex.sub(r"\1\1", tweet)

def extractHashtags(tweet):
    return ' '.join([x['text'] for x in tweet.get('entities',{}).get('hashtags')]).lower()

def checkEmoticons(attitude,tweet_text):
    found = False
    for emoticon in config.get('datasets',{}).get('emoticons',{}).get(attitude,{}).get('replace',[]):
        if emoticon in tweet_text:
            found = True 
    if not found:
        print tweet_text
        
def removeEmoticons(tweet):
    """
    Removes the emoticons from the tweets.
    E.g. "I really like it :-)" => "I really like it"
    Does not handle emoticons like ":-))))))" but those will be removed in a final step after all preprocessing
    """
    return re.sub(r'(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)','',tweet)

def apostropheLookup(tweet):
    """
    Maintains a common usage of apostophes
    E.g. "I've been happy" => "I have been happy"
    First try to find the exact contraction in the wikipedia list, and then try to replace by smaller apostrophe dict
    """
    tweet = ' '.join(contractions[word] if word in contractions else word for word in tweet.split())
    tokens = nltk.word_tokenize(tweet)
    return ' '.join([apostropheDictionary[token] if token in apostropheDictionary else token for token in tokens])

def replaceSlang(tweet):
    """
    Replaces the slang in the tweet based on the previously generated dictionary from a online website describing
    the different slang words.
    Replacement is done by the written out slang
    But only replace if the token length is min 2 characters long => do not replace numbers, questionmarks, etc.
    """
    tokens = tweet.split(' ')
    return ' '.join([slangDict[token.lower()] if token.lower() in slangDict and len(token)>2 else token for token in tokens])

def replaceTwitterSlang(tweet):
    """
    Replaces twitter specific slang, based on a dictionary created from various online sources
    """
    tokens = tweet.split(' ')
    return ' '.join([slangTwitter[token.lower()] if token.lower() in slangTwitter else token for token in tokens])

#matches 1114 emojis, taken from http://stackoverflow.com/questions/43242440/javascript-unicode-emoji-regular-expressions
emojiRegex = r'(?:[\u00A9\u00AE\u203C\u2049\u2122\u2139\u2194-\u2199\u21A9-\u21AA\u231A-\u231B\u2328\u23CF\u23E9-\u23F3\u23F8-\u23FA\u24C2\u25AA-\u25AB\u25B6\u25C0\u25FB-\u25FE\u2600-\u2604\u260E\u2611\u2614-\u2615\u2618\u261D\u2620\u2622-\u2623\u2626\u262A\u262E-\u262F\u2638-\u263A\u2640\u2642\u2648-\u2653\u2660\u2663\u2665-\u2666\u2668\u267B\u267F\u2692-\u2697\u2699\u269B-\u269C\u26A0-\u26A1\u26AA-\u26AB\u26B0-\u26B1\u26BD-\u26BE\u26C4-\u26C5\u26C8\u26CE-\u26CF\u26D1\u26D3-\u26D4\u26E9-\u26EA\u26F0-\u26F5\u26F7-\u26FA\u26FD\u2702\u2705\u2708-\u270D\u270F\u2712\u2714\u2716\u271D\u2721\u2728\u2733-\u2734\u2744\u2747\u274C\u274E\u2753-\u2755\u2757\u2763-\u2764\u2795-\u2797\u27A1\u27B0\u27BF\u2934-\u2935\u2B05-\u2B07\u2B1B-\u2B1C\u2B50\u2B55\u3030\u303D\u3297\u3299]|(?:\uD83C[\uDC04\uDCCF\uDD70-\uDD71\uDD7E-\uDD7F\uDD8E\uDD91-\uDD9A\uDDE6-\uDDFF\uDE01-\uDE02\uDE1A\uDE2F\uDE32-\uDE3A\uDE50-\uDE51\uDF00-\uDF21\uDF24-\uDF93\uDF96-\uDF97\uDF99-\uDF9B\uDF9E-\uDFF0\uDFF3-\uDFF5\uDFF7-\uDFFF]|\uD83D[\uDC00-\uDCFD\uDCFF-\uDD3D\uDD49-\uDD4E\uDD50-\uDD67\uDD6F-\uDD70\uDD73-\uDD7A\uDD87\uDD8A-\uDD8D\uDD90\uDD95-\uDD96\uDDA4-\uDDA5\uDDA8\uDDB1-\uDDB2\uDDBC\uDDC2-\uDDC4\uDDD1-\uDDD3\uDDDC-\uDDDE\uDDE1\uDDE3\uDDE8\uDDEF\uDDF3\uDDFA-\uDE4F\uDE80-\uDEC5\uDECB-\uDED2\uDEE0-\uDEE5\uDEE9\uDEEB-\uDEEC\uDEF0\uDEF3-\uDEF6]|\uD83E[\uDD10-\uDD1E\uDD20-\uDD27\uDD30\uDD33-\uDD3A\uDD3C-\uDD3E\uDD40-\uDD45\uDD47-\uDD4B\uDD50-\uDD5E\uDD80-\uDD91\uDDC0]))'
emoji_pattern = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\ud83c[\udde0-\uddff])|"  # flags (iOS)
    u"(\udf00-\uffff)"           #wrong
    "+")
def extractEmojis(tweet):
    """
    Extracts emojies from tweet
    """
    emojies = emoji_pattern.findall(tweet)
    #put all in array
    return [emoji for sublist in [list(emoji) for emoji in emojies] for emoji in sublist if emoji!='']

def splitEmojis(tweet):
    """
    Sometimes emojies are concanated and the tokenizer can not split them
    This function inserts spaces before and after each emoji
    """
    emojies = extractEmojis(tweet)
    for emoji in emojies:
        tweet = tweet.replace(emoji,' '+emoji+' ')
        #remove wrong emoji
        if emoji==u'\ud83c' or emoji==u'\uddfa':
            tweet = tweet.replace(emoji,'')
    return tweet

def spellingCorrection(tweet,whitelist):
    """
    Correction of tweet spelling.
    E.g. "speling" => "spelling"
    Only if the word is longer than 3 characters=> do not try to correct abbreviations or other small words
    And only if the word does not appear within the top 50% of words => do not change iphone to phone, etc.
    """
    tokens = nltk.word_tokenize(tweet)
    try:
        correction = ' '.join([word if dictionary.check(word) else dictionary.suggest(word)[0] if len(word)>3 and word not in whitelist and not word.startswith("xx") and not (word.startswith('\u') or word.startswith('\U')) and len(dictionary.suggest(word))>0 else word for word in tokens])
        return correction
    except:
        return tweet

translate_table = dict((ord(char), None) for char in string.punctuation)
def removePunctuationAndMisc(tweet):
    """
    Removes all the punctuation from the string and clean duplicate spaces, strip, etc.
    E.g. "... i dont like it" => "i dont like it"
    """
    return re.sub("\s\s+" , " ", ''.join([i for i in tweet.translate(translate_table) if not i.isdigit()])).strip()


def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

lmtzr = WordNetLemmatizer()
def lemmatize_tweet(tweet):
    """
    Lemmatizes the given tweet using nltk WordNetLemmatizer
    """
    tokens = nltk.word_tokenize(tweet)
    pos_tagged_words = pos_tag(tokens)
    lemmatized_tweet = [lmtzr.lemmatize(i[0],get_wordnet_pos(i[1])) for i in pos_tagged_words]
    return ' '.join(lemmatized_tweet).strip()

def replaceEntities(tweet,topic):
    """
    Replaces twitter specific entities like hashtags, mentions, 
    urls, etc. and replaces them by static string
    """
    #pprint.pprint(tweet)
    temp_indices = {}
    processed_tweet = tweet.get('full_text')
    
    for entity_name,entity_object in tweet.get('entities',{}).iteritems():
        if entity_object:
            replace_string = " XX"+entity_name.upper()+"XX "
            for entity in entity_object:
                indices = entity.get('indices')
                temp_indices[indices[0]] = (replace_string,indices[1])
    #replace the entities backwards in order to maintain the indices
    for position in sorted(temp_indices.iterkeys(),reverse=True):
        processed_tweet = processed_tweet[:position]+temp_indices[position][0]+processed_tweet[temp_indices[position][1]:]


    #replace topic, only whole words
    processed_tweet = re.sub(r'\b%s\b' % topic, ' XXTOPICXX ',processed_tweet,flags=re.I)
    return processed_tweet

In [40]:
print(len(data))

123457


In [90]:
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))


#datasubet = take(10, data.iteritems())


from ipywidgets import FloatProgress
from IPython.display import display
progress = FloatProgress(min=0, max=len(data),description="Processing the text")
display(progress)



dictionary = enchant.Dict("en_US")
#helper dictionary to count the corpus size after each processing step
corpus_tokens = {
    "00_raw":{
        "tokens":set(),
        "touched":0
    },
    "01_replaceEntities":{
        "tokens":set(),
        "touched":0
    },
    "02_misc":{
        "tokens":set(),
        "touched":0
    },
    "03_removeEmoticons":{
        "tokens":set(),
        "touched":0
    },
    "04_removeDuplicatedCharacters":{
        "tokens":set(),
        "touched":0
    },
    "05_apostropheLookup":{
        "tokens":set(),
        "touched":0
    },
    "06_replaceSlang":{
        "tokens":set(),
        "touched":0
    },
    "07_replaceTwitterSlang":{
        "tokens":set(),
        "touched":0
    },
    "08_splitEmojis":{
        "tokens":set(),
        "touched":0
    },
    "09_removePunctuationAndMisc":{
        "tokens":set(),
        "touched":0
    },
    "10_spellingCorrection":{
        "tokens":set(),
        "touched":0
    },
    "11_lemmatization":{
        "tokens":set(),
        "touched":0
    }
}


def didChange(oldtweet,newtweet):
    if oldtweet==newtweet:
        return 0
    else:
        return 1

for tweet_id,tweet in data.iteritems():
    progress.value += 1
    
    
    corpus_tokens['00_raw']['tokens'].update(nltk.word_tokenize(tweet['full_text']))
    
    tweet["extracted_hashtags"] = extractHashtags(tweet)
    
    #replace entities with placeholders
    replace_entities_text = tweet["processed_tweet"] = replaceEntities(tweet,tweet['category']['seed_topic'])
    corpus_tokens['01_replaceEntities']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['01_replaceEntities']['touched'] += didChange(tweet['full_text'],tweet["processed_tweet"])
    
    #Misc preprocessing
    tweet["processed_tweet"] = tweet["processed_tweet"].lower()
    misc_text = tweet["processed_tweet"] = removeLinebreaks(tweet["processed_tweet"])
    corpus_tokens['02_misc']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['02_misc']['touched'] += didChange(replace_entities_text,tweet["processed_tweet"])
    
    #remove emoticons from tweets
    removeEmoticonText = tweet["processed_tweet"] = removeEmoticons(tweet["processed_tweet"])
    corpus_tokens['03_removeEmoticons']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['03_removeEmoticons']['touched'] += didChange(misc_text,tweet["processed_tweet"])
    
    #remove duplicated characters
    removeDuplicateText = tweet["processed_tweet"] = removeDuplicatedCharacters(tweet["processed_tweet"])
    corpus_tokens['04_removeDuplicatedCharacters']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['04_removeDuplicatedCharacters']['touched'] += didChange(removeEmoticonText,tweet["processed_tweet"])
    
    #remove apostrophes and replace by written word
    apostrophe_text = tweet["processed_tweet"] = apostropheLookup(tweet["processed_tweet"])
    corpus_tokens['05_apostropheLookup']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['05_apostropheLookup']['touched'] += didChange(removeDuplicateText,tweet["processed_tweet"])
    
    #replace slang
    replaceslang_text = tweet["processed_tweet"] = replaceSlang(tweet["processed_tweet"])
    corpus_tokens['06_replaceSlang']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['06_replaceSlang']['touched'] += didChange(apostrophe_text,tweet["processed_tweet"])
    
    #replace twitter speicifc slang
    replacetwitterslang_text = tweet["processed_tweet"] = replaceTwitterSlang(tweet["processed_tweet"])
    corpus_tokens['07_replaceTwitterSlang']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['07_replaceTwitterSlang']['touched'] += didChange(replaceslang_text,tweet["processed_tweet"])
    
    #split emojis
    splitemoji_text = tweet["processed_tweet"] = splitEmojis(tweet["processed_tweet"])
    corpus_tokens['08_splitEmojis']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['08_splitEmojis']['touched'] += didChange(replacetwitterslang_text,tweet["processed_tweet"])
    
    #remove punctuation
    tweet["processed_tweet"] = removePunctuationAndMisc(tweet["processed_tweet"])
    corpus_tokens['09_removePunctuationAndMisc']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['09_removePunctuationAndMisc']['touched'] += didChange(splitemoji_text,tweet["processed_tweet"])
    
spelling_progress = FloatProgress(min=0, max=len(data),description="Spellingcorrection and Lemmatization")
display(spelling_progress)

#Create a corpus of all words in the tweets
#Just as a helper for other functions
tweet_dictionary = [token for sublist in [nltk.word_tokenize(tweet["processed_tweet"]) for tweet_id,tweet in data.iteritems()] for token in sublist]
tweet_dictionary_counter = Counter(tweet_dictionary)
most_common_words = [word for word,count in tweet_dictionary_counter.items() if count<4]
#perform a second loop over the processed_tweets because the dictionary has to be created on the complete data

for tweet_id,tweet in data.iteritems():
    spelling_progress.value += 1
    #spelling at the end, after the tweets have been processed
    old_text = tweet["processed_tweet"]
    spellcorrection_text = tweet["processed_tweet"] = spellingCorrection(tweet["processed_tweet"],whitelist=most_common_words)
    corpus_tokens['10_spellingCorrection']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['10_spellingCorrection']['touched'] += didChange(old_text,tweet["processed_tweet"])

    #lemmatization
    tweet["processed_tweet"] = lemmatize_tweet(tweet["processed_tweet"])
    corpus_tokens['11_lemmatization']['tokens'].update(nltk.word_tokenize(tweet["processed_tweet"]))
    corpus_tokens['11_lemmatization']['touched'] += didChange(spellcorrection_text,tweet["processed_tweet"])

counter = {}
for key,value in corpus_tokens.iteritems():
    counter[key] = {
        "counter":len(value['tokens']),
        "touched":value['touched']
    }
pprint(counter)

# Export Final Preprocessed Tweets

In [91]:
export = [{
    "id":tweet_id,
    "hashtags":tweet['extracted_hashtags'],
    "raw_text":removeLinebreaks(tweet['full_text']),
    "processed_text":tweet['processed_tweet'],
    "topic":tweet['category']['top_topic'],
    "seed_topic":tweet['category']['seed_topic'],
    "attitude":tweet['category']['attitude']
} for tweet_id,tweet in data.iteritems()]
df_export = pd.DataFrame.from_dict(export)
df_export.to_csv("./data/processed/final_dataset.csv",encoding="utf-8")

## Create Subset of Data for Manual Labelling
One percent of the complete data set will be sampled.
The distribution of topic and attitude is taken into account in oder to maintain the same characteristics for the subset dataset

In [110]:
tempdf = df_export.groupby(['topic','attitude']).size().reset_index(name="total_count")
tempdf = tempdf.merge(df_export.groupby('topic').size().reset_index(name="total_topic"),how="left",on="topic")
tempdf['amount_to_select'] = (tempdf['total_count']*0.01).astype('int')

df_subset = pd.DataFrame()
#make the subset selection
for index, row in tempdf.iterrows():
    df_subset = pd.concat([df_subset,df_export[(df_export['topic']==row['topic']) &(df_export['attitude']==row['attitude']) ].sample(n=row['amount_to_select'],random_state=4)])

df_subset.to_csv("./data/processed/final_dataset_subset.csv",encoding="utf-8")